<a href="https://colab.research.google.com/github/stefano2211/DataScienceRedSocial/blob/main/YummyAnalisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Importamos las librerias
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet
from textblob import TextBlob
import snscrape.modules.twitter as sntwitter
import plotly.express as px
import neattext as nt
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
#Cargamos las stopwords de nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
#Importamos los datos de twitter
query = "yummy venezuela"
tweets = []
limit = 2000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.content, tweet.hashtags, tweet.username])

<ipython-input-8-6e09f988df88>:13: FutureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.content, tweet.hashtags, tweet.username])
<ipython-input-8-6e09f988df88>:13: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.content, tweet.hashtags, tweet.username])


In [9]:
#Colocamos los datos en un dataframe
dt = pd.DataFrame(tweets, columns=['Date', 'Tweet', 'Hashtag', 'Username'])
dt.head(100)

,Date,Tweet,Hashtag,Username
0,2023-02-12 14:43:07+00:00,Este no es el primer acercamiento de Yummy con...,[Criptolugares],criptolugares
1,2023-02-12 14:43:05+00:00,Yummy lanza colección de NFT para financiar st...,[Criptolugares],criptolugares
2,2023-02-12 05:04:54+00:00,@MilAbdianaM @yummy_superapp Cuando estaba en ...,None,nany_mavare94
3,2023-02-12 00:38:17+00:00,Yummy Venezuela es una mierda,None,mariamartiochoa
4,2023-02-11 21:26:00+00:00,@tusambil Sambil Candelaria es territorio INK\...,"[TATUAJES, VENEZUELA, INK, CARACAS, YUMMY]",guacamayaink
...,...,...,...,...
95,2022-12-28 22:45:30+00:00,Bloomberg: Yummy incrementará la inversión en ...,None,descifradocom
96,2022-12-28 21:10:00+00:00,Yummy incrementará la inversión en Venezuela e...,[Economía],ElAragueno
97,2022-12-28 12:40:30+00:00,Yummy incrementará la inversión en Venezuela e...,[Economía],ElAragueno
98,2022-12-28 00:15:30+00:00,Bloomberg: Yummy incrementará la inversión en ...,None,descifradocom


In [10]:
#Convertimos la columna Date a formato tiempo
dt['Date'] = pd.to_datetime(dt['Date'], format='%Y%m%d')

In [11]:
#Convertimos el None en Nan y luego remplazamos el Nan por un 1
dt['Hashtag'].replace('None', np.nan, inplace=True)
dt['Hashtag'] = dt['Hashtag'].fillna('1')

In [12]:
dt.head()

,Date,Tweet,Hashtag,Username
0,2023-02-12 14:43:07+00:00,Este no es el primer acercamiento de Yummy con...,[Criptolugares],criptolugares
1,2023-02-12 14:43:05+00:00,Yummy lanza colección de NFT para financiar st...,[Criptolugares],criptolugares
2,2023-02-12 05:04:54+00:00,@MilAbdianaM @yummy_superapp Cuando estaba en ...,1,nany_mavare94
3,2023-02-12 00:38:17+00:00,Yummy Venezuela es una mierda,1,mariamartiochoa
4,2023-02-11 21:26:00+00:00,@tusambil Sambil Candelaria es territorio INK\...,"[TATUAJES, VENEZUELA, INK, CARACAS, YUMMY]",guacamayaink


In [13]:
#Extraemos los hashtags para convertirlos en strings
def textl(text):
  leters= " ".join([str(_) for _ in text])
  return leters


In [14]:
dt['Token'] = dt['Hashtag'].apply(textl)
dt.head()

,Date,Tweet,Hashtag,Username,Token
0,2023-02-12 14:43:07+00:00,Este no es el primer acercamiento de Yummy con...,[Criptolugares],criptolugares,Criptolugares
1,2023-02-12 14:43:05+00:00,Yummy lanza colección de NFT para financiar st...,[Criptolugares],criptolugares,Criptolugares
2,2023-02-12 05:04:54+00:00,@MilAbdianaM @yummy_superapp Cuando estaba en ...,1,nany_mavare94,1
3,2023-02-12 00:38:17+00:00,Yummy Venezuela es una mierda,1,mariamartiochoa,1
4,2023-02-11 21:26:00+00:00,@tusambil Sambil Candelaria es territorio INK\...,"[TATUAJES, VENEZUELA, INK, CARACAS, YUMMY]",guacamayaink,TATUAJES VENEZUELA INK CARACAS YUMMY


In [15]:
tokenHashtag = dt['Token']
tokenHashtag

0                              Criptolugares
1                              Criptolugares
2                                          1
3                                          1
4       TATUAJES VENEZUELA INK CARACAS YUMMY
                        ...                 
1995                                       1
1996                                       1
1997                                       1
1998                                       1
1999                                       1
Name: Token, Length: 2000, dtype: object

In [16]:
tokens = [token for line in tokenHashtag  for token in line.split()]

In [17]:
#Creamos una funcion que cuente los datos ya toquenizados 
def get_tokens(docx,num=40):
    word_tokens = Counter(docx)
    most_common = word_tokens.most_common(num)
    result = dict(most_common)
    return result

In [18]:
tokensTotal = get_tokens(tokens)

In [19]:
#Insertamos los datos en un dataframe
data = pd.DataFrame(tokensTotal.items(),columns=['Hashtags','scores'])
data

,Hashtags,scores
0,1,1432
1,Venezuela,206
2,yummy,76
3,NegociosHispanoPost,70
4,venezuela,55
5,Yummy,50
6,Caracas,27
7,delivery,19
8,caracas,19
9,Viral,18


In [20]:
#Ploteamos los datos
fig = px.bar(data, title="Hashtags mas usados en Yummy Venezuela",x=data['Hashtags'], y=data['scores'])
fig.show()

#Conclusiones de los hashtags

Los hashtags mas utilizados de Yummy Venezuela


*   Yummy
*   NegociosHispanosPost
*   Venezuela
*   Caracas
*   Delivery



In [21]:
#Removemos caracteres que puedan afectar la prediccion del modelo
dt['Tweet'] = dt['Tweet'].apply(nt.remove_hashtags)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_emojis)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_numbers)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_special_characters)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_urls)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_currency_symbols)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_userhandles)

In [22]:
#Creamos una funcion para borrar caracteres especiales
def clean_text(text):
  text = re.sub(r'@[A-Za-z09]+', '', text)
  text = re.sub(r'#', '', text)
  text = re.sub(r'RT[\s]+', '', text)
  text = re.sub(r'https?:\/\/?', '', text)


  return text

dt['Tweet'] = dt['Tweet'].apply(clean_text)

In [23]:
#Creamos dos fuciones una para clacular el sentimiento y el otro para calcular la subjetividad
def get_subjectivity(text):
  return TextBlob(text).translate(from_lang="es", to="en").sentiment.subjectivity

def get_polarity(text):
  return TextBlob(text).translate(from_lang="es", to="en").sentiment.polarity

In [24]:
dt['Subjectivity'] = dt['Tweet'].apply(get_subjectivity)

In [25]:
dt['Polarity'] = dt['Tweet'].apply(get_polarity)

In [26]:
#Creamos una funcion que tome la polaridad y la procese como Positivo, Neutro o negativo
def analysis(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
     return 'Neutral'
  elif score > 0:
    return 'Positive'

dt['Analysis'] = dt['Polarity'].apply(analysis)

dt.head(800)

,Date,Tweet,Hashtag,Username,Token,Subjectivity,Polarity,Analysis
0,2023-02-12 14:43:07+00:00,Este no es el primer acercamiento de Yummy con...,[Criptolugares],criptolugares,Criptolugares,0.333333,0.25,Positive
1,2023-02-12 14:43:05+00:00,Yummy lanza coleccin de NFT para financiar sta...,[Criptolugares],criptolugares,Criptolugares,0.300000,-0.20,Negative
2,2023-02-12 05:04:54+00:00,MilAbdianaM yummysuperapp Cuando estaba en Ven...,1,nany_mavare94,1,0.783333,-0.65,Negative
3,2023-02-12 00:38:17+00:00,Yummy Venezuela es una mierda,1,mariamartiochoa,1,0.800000,-0.20,Negative
4,2023-02-11 21:26:00+00:00,tusambil Sambil Candelaria es territorio INK ...,"[TATUAJES, VENEZUELA, INK, CARACAS, YUMMY]",guacamayaink,TATUAJES VENEZUELA INK CARACAS YUMMY,0.000000,0.00,Neutral
...,...,...,...,...,...,...,...,...
795,2022-04-14 22:51:51+00:00,Desde este de abril la aplicacin de entregas ...,1,glendagrisell,1,0.000000,0.00,Neutral
796,2022-04-14 22:41:09+00:00,Servicio de taxis Yummy integra pagos con Bina...,[Adopción],CriptoNoticias,Adopción,0.000000,0.00,Neutral
797,2022-04-14 22:38:33+00:00,Le service de taxi Yummy intgre les paiements ...,1,LaCryptoMonnai1,1,0.000000,0.00,Neutral
798,2022-04-14 22:36:52+00:00,Servicio de taxis Yummy integra pagos con Bina...,1,gentecosmotv,1,0.000000,0.00,Neutral


In [27]:
count = dt['Analysis'].value_counts()
count

Neutral     1039
Positive     725
Negative     236
Name: Analysis, dtype: int64

In [28]:
#Graficamos la polaridad
fig = px.scatter(dt, y=dt['Polarity'])
fig.show()

In [29]:
fig = px.bar(count, title="Sentimientos de los tweets de Yummy Venezuela", color=['red','blue','green'])
fig.show()

#Conclusiones de el analisis de emociones

Podemos concluir que hay mas comentarios negativos que positivos pero lo que mas se resalta es los comentarios neutros con una alta diferencia.



1.   Neutros: 1039
2.   Negativos: 236
3.   Positivos: 725


In [33]:
# Creamos tres sets de datos con los diferentes tipos de comentarios
positive_tweet = dt[dt['Analysis'] == 'Positive']
neutral_tweet = dt[dt['Analysis'] == 'Neutral']
negative_tweet = dt[dt['Analysis'] == 'Negative']

In [34]:
print(stopwords.words('spanish'))

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

In [36]:
#Eliminamos las preposiciones, adverbios y adjetivos
def textdrop(text):

  sw = stopwords.words('spanish')

  text = [word.lower() for word in text.split() if word.lower() not in sw]

  return " ".join(text)

In [ ]:
positive_tweet['Tweet'] = positive_tweet['Tweet'].apply(textdrop)
neutral_tweet['Tweet'] = neutral_tweet['Tweet'].apply(textdrop)
negative_tweet['Tweet'] = negative_tweet['Tweet'].apply(textdrop)

In [38]:
tokenPositive = positive_tweet['Tweet']
tokenNeutral = neutral_tweet['Tweet']
tokenNegative = negative_tweet['Tweet']

In [39]:
#Tokenizamos los tweets
pos_tokens = [token for line in tokenPositive  for token in line.split()]
neg_tokens = [token for line in tokenNeutral  for token in line.split()]
neut_tokens = [token for line in tokenNegative  for token in line.split()]

In [40]:
word_pos = get_tokens(pos_tokens)
word_neu = get_tokens(neut_tokens)
word_neg = get_tokens(neg_tokens)

In [41]:
#Creamos 3 sets de datos con las palabras y el contador 
pos_df = pd.DataFrame(word_pos.items(),columns=['words','scores'])
neut_df = pd.DataFrame(word_neu.items(),columns=['words','scores'])
neg_df = pd.DataFrame(word_neg.items(),columns=['words','scores'])

In [42]:
fig = px.bar(data, title="Palabras mas usadas en los tweets positivos de Yummy Venezuela",x=pos_df['words'], y=pos_df['scores'])
fig.show()

In [44]:
fig = px.bar(data, title="Palabras mas usadas en los tweets negativos de Yummy Venezuela",x=neg_df['words'], y=neg_df['scores'])
fig.show()

#Conclusiones de la tokenzacion

Podemos observar que en los tweets positivos las palabras claves que se repiten comunmente es:

*   Yummy
*   Venezuela
*   App
*   Rides

En los tweets negativo los mas resaltantes son:

*   Yummy
*   Venezuela
*   Delivery
*   Visionario

In [ ]:
neg = negative_tweet.loc[negative_tweet.Tweet.str.contains('delivery')]
neg.head()

In [63]:
neg.shape

(27, 8)

#Aspecto resaltante

De los 236 comentarios negativos 27 son relacionados con la palabra delivery

In [64]:
fig = px.histogram(dt, x=dt["Date"], title="Tweets durante durante el tiempo")
fig.update_layout(bargap=0.2)
fig.show()

#Tweets durante el Tiempo

Podemos ver que hay tres picos de comentarios que abarcan desde el 1 de abril de 2021 hasta 1 de febrero de 2023



1.   De Julio 1 a Agosto 1 de 2021
2.   De Mayo 1 a Junio 1 de 2021
3.   De Septiembre 1 a Octubre 1 de 2022



In [65]:
fig = px.histogram(dt, x=positive_tweet["Date"], title="Tweets positivos durante el tiempo")
fig.update_layout(bargap=0.2)
fig.show()

#Tweets Positivos durante el tiempo

Podemos observar dos puntos principales 

1.   En el mes de Julio de 2021
2.   En el mes de Septiembre de 2021



In [66]:
fig = px.histogram(dt, x=negative_tweet["Date"], title="Tweets negativos durante el tiempo")
fig.update_layout(bargap=0.2)
fig.show()

#Tweets Negativos durante el tiempo

Podemos observar dos puntos principales 

1.   En el mes de Julio de 2022
2.   En el mes de Septiembre de 2022